## Explorando dados para uma Análise de Credit Score

Neste notebook vamos analisar/explorar dados do Banco Central do Brasil do Sistema Financeiro Nacional. 
Base de dados de 05/2024


Metodologia disponível no Portal de Dados Abertos do Bacen:
https://www.bcb.gov.br/content/estabilidadefinanceira/scr/scr.data/scr_data_metodologia.pdf


In [1]:
# Importando as bibliotecas que serão utilizadas para o processo de ETL

import pandas as pd
import glob as gl
from random import randint
import plotly.express as px

#### Criando a função que lê o arquivo .csv referente a 05/2024, gravados no mesmo diretório do notebook.

In [2]:
def read_code(caminho_dataset):
     """Esta função recebe como parâmetro o caminho dos arquivos no formato '*.csv' lê todos os arquivos.csv do diretório dataset que 
        contém os dados correspondentes a cada database e concatena em um só dataframe.
        1- Cria uma lista com os nomes dos arquivos do dataset.
        2- Cria uma lista vazia para armazenar os dataframes.
        3- Usa uma estrutura de repetição para ler cada arquivo e armazenar na lista dfs, importando apenas as colunas que serão usadas.
        4- Concatena os dataframes em um só, o comando concat precisa receber com parâmetro a lista de dfs
     """
     arquivos = gl.glob(caminho_dataset, recursive=True)
     dfs=[]
     for arquivo in arquivos:
          df = pd.read_csv(arquivo, sep=';', 
                                    encoding='utf-8', 
                                    thousands='.', 
                                    decimal=',')
          dfs.append(df)
     df= pd.concat(dfs, ignore_index=True)
     return df

#### Criando a função que limpa espaços vazios e strings redundantes de algumas colunas

In [3]:
def clean_code(df):    
     """Esta função realiza a limpeza dos dados do dataframe:

     1- Extrai os espaços vazios da coluna 'cliente'
     2- Extrai os espaços vazios da coluna 'uf'
     3- Extrai a string PF - da coluna ocupacao 
     4- Extrai a string PJ - da coluna cnae_secao
     5- Extrai a string PJ - da coluna cnae_subclasse
     6- Extrai a string PF - da coluna porte
     7- Extrai a string PJ - da coluna porte
     8- Extrai a string PF - da coluna modalidade
     9- Extrai a string PJ - da coluna modalidade
     
     """
     df.loc[:, 'cliente']= df.loc[:, 'cliente'].str.strip()
     df.loc[:, 'uf']= df.loc[:, 'uf'].str.strip()
     df.loc[:, 'ocupacao']= df.loc[:, 'ocupacao'].str.strip('PF - ')
     df.loc[:, 'cnae_secao']= df.loc[:, 'cnae_secao'].str.strip('PJ - ')
     df.loc[:, 'cnae_subclasse']= df.loc[:, 'cnae_subclasse'].str.strip('PJ - ')
     df.loc[:, 'porte']= df.loc[:, 'porte'].str.strip('PF - ')
     df.loc[:, 'porte']= df.loc[:, 'porte'].str.strip('J - ')
     df.loc[:, 'modalidade']= df.loc[:, 'modalidade'].str.strip('PF - ')
     df.loc[:, 'modalidade']= df.loc[:, 'modalidade'].str.strip('PJ - ')

     return df

#### Criando a função que ajusta os tipos de dados de string para numérico e agrupa os atributos ocupação (PF) e CNAE (PJ)

In [4]:
def change_code(df):
     """ 
     Esta função ajusta os tipos de dados de acordo com a informação contida nele:

     1 - Alterando a coluna data-base para tipo datetime         
     2 - Ajustando as colunas 'ativo_problematico' e 'carteira_inadimplida_arrastada' para tipo float
     3 - Ajustando a Coluna Ocupacao para os cliente PJ (substituindo pelos dados da coluna cnae_secao - Atividade principal)
     4 - Usando random pra atribuir um número entre 1 e 15 para número de operações (quando são uma string <=15)    
     5 - Ajusta o tipo de dados de string (object) para inteiro na coluna numero de operacoes
     
     """
     df['data_base'] = pd.to_datetime(df['data_base']) 
     df['carteira_inadimplida_arrastada']=df['carteira_inadimplida_arrastada'].astype(float, copy=False)
     df.loc[df['cliente']=='PJ', 'ocupacao']= df['cnae_secao']
     df.loc[df['numero_de_operacoes'] == '<= 15', 'numero_de_operacoes'] = (df.loc[df['numero_de_operacoes'] == '<= 15', 'numero_de_operacoes'].apply(lambda x: randint(1, 15) if x.strip() == '<= 15' else x))
     df['numero_de_operacoes'] = df['numero_de_operacoes'].astype(int)
     return df

#### Chamando a função de leitura dos dados 

In [5]:
# ---------------------------------------------- Informando o caminho do dataset, o diretório que contém todos os arquivos da data-base 2023 --------#.
caminho_dataset= './planilha_202205.csv'
df = read_code(caminho_dataset)

#### Chamando a função de limpeza dos dados

In [6]:
# ---------------------------------------------- Chamando a função de limpeza dos dados -------------------------------------------------------------#
df= clean_code(df)

#### Chamando a função que ajusta os tipos de dados

In [7]:
# ---------------------------------------------- Chamando a função que ajusta os tipos de dados ------------------------------------------------------------------------#
df= change_code(df)

In [8]:
df['data_base'].head(1)

0   2022-05-31
Name: data_base, dtype: datetime64[ns]

In [9]:
df1=df
df1['porte'].unique()

array(['Mais de 1 a 2 salários mínimos', 'Sem rendimento',
       'Mais de 2 a 3 salários mínimos', 'Mais de 3 a 5 salários mínimos',
       'Até 1 salário mínimo', 'Mais de 5 a 10 salários mínimos', 'Micro',
       'Acima de 20 salários mínimos', 'Indisponível',
       'Mais de 10 a 20 salários mínimos', 'Pequeno', 'Grande', 'Médio'],
      dtype=object)

In [10]:
# ---------------------------------------------- Média das operações de PJ  ---------------------------------------------------------------------------------------#

med_pj=(df1.query('cliente=="PJ"').carteira_ativa.sum())/(df1.query('cliente=="PJ"').numero_de_operacoes.sum())
print( 'Valor médio de operações de Pessoa Jurídica do País: R$', '{:,.2f}'.format(med_pj))

med_pf=(df1.query('cliente=="PF"').carteira_ativa.sum())/(df1.query('cliente=="PF"').numero_de_operacoes.sum())
print( 'Valor médio de operações de Pessoa Física do País: R$', '{:,.2f}'.format(med_pf))

Valor médio de operações de Pessoa Jurídica do País: R$ 65,147.42
Valor médio de operações de Pessoa Física do País: R$ 5,658.54


In [11]:
#Cartão de Crédito PF
med_cc_pf=(df1.query('modalidade=="Cartão de crédito" and cliente=="PF"').carteira_ativa.sum())/(df1.query('modalidade=="Cartão de crédito"'and 'cliente=="PF"').numero_de_operacoes.sum())
print( 'Valor médio de operações com Cartão de Crédito de Pessoa Física no País: R$', '{:,.2f}'.format(med_cc_pf))


cond_1SM='modalidade=="Cartão de crédito" and cliente=="PF" and porte=="Até 1 salário mínimo"'
cond_2SM='modalidade=="Cartão de crédito" and cliente=="PF" and porte=="Mais de 1 a 2 salários mínimos"'
cond_3SM='modalidade=="Cartão de crédito" and cliente=="PF" and porte=="Mais de 2 a 3 salários mínimos"'
cond_5SM='modalidade=="Cartão de crédito" and cliente=="PF" and porte=="Mais de 3 a 5 salários mínimos"'
cond_10SM='modalidade=="Cartão de crédito" and cliente=="PF" and porte=="Mais de 5 a 10 salários mínimos"'
cond_20SM='modalidade=="Cartão de crédito" and cliente=="PF" and porte=="Mais de 10 a 20 salários mínimos"'
cond_A20SM='modalidade=="Cartão de crédito" and cliente=="PF" and porte=="Acima de 20 salários mínimos"'

med=(df1.query(cond_1SM).carteira_ativa.sum())/(df1.query(cond_1SM).numero_de_operacoes.sum())
print( 'Valor médio de operações com Cartão de Crédito de Pessoa Física no País com renda até 1 Salário Mínimo: R$', '{:,.2f}'.format(med))

med=(df1.query(cond_2SM).carteira_ativa.sum())/(df1.query(cond_2SM).numero_de_operacoes.sum())
print( 'Valor médio de operações com Cartão de Crédito de Pessoa Física no País com renda de 1 a 2 salários mínimos: R$', '{:,.2f}'.format(med))

med=(df1.query(cond_3SM).carteira_ativa.sum())/(df1.query(cond_3SM).numero_de_operacoes.sum())
print( 'Valor médio de operações com Cartão de Crédito de Pessoa Física no País com renda Mais de 2 a 3 salários mínimos: R$', '{:,.2f}'.format(med))

med=(df1.query(cond_5SM).carteira_ativa.sum())/(df1.query(cond_5SM).numero_de_operacoes.sum())
print( 'Valor médio de operações com Cartão de Crédito de Pessoa Física no País com renda Mais de 3 a 5 salários mínimos: R$', '{:,.2f}'.format(med))

med=(df1.query(cond_10SM).carteira_ativa.sum())/(df1.query(cond_10SM).numero_de_operacoes.sum())
print( 'Valor médio de operações com Cartão de Crédito de Pessoa Física no País com renda Mais de 5 a 10 salários mínimos: R$', '{:,.2f}'.format(med))

med=(df1.query(cond_20SM).carteira_ativa.sum())/(df1.query(cond_20SM).numero_de_operacoes.sum())
print( 'Valor médio de operações com Cartão de Crédito de Pessoa Física no País com renda Mais de 10 a 20 salários mínimos: R$', '{:,.2f}'.format(med))

med=(df1.query(cond_A20SM).carteira_ativa.sum())/(df1.query(cond_A20SM).numero_de_operacoes.sum())
print( 'Valor médio de operações com Cartão de Crédito de Pessoa Física no País com renda Acima de 20 salários mínimos: R$', '{:,.2f}'.format(med))


Valor médio de operações com Cartão de Crédito de Pessoa Física no País: R$ 852.92
Valor médio de operações com Cartão de Crédito de Pessoa Física no País com renda até 1 Salário Mínimo: R$ 546.42
Valor médio de operações com Cartão de Crédito de Pessoa Física no País com renda de 1 a 2 salários mínimos: R$ 804.16
Valor médio de operações com Cartão de Crédito de Pessoa Física no País com renda Mais de 2 a 3 salários mínimos: R$ 1,085.67
Valor médio de operações com Cartão de Crédito de Pessoa Física no País com renda Mais de 3 a 5 salários mínimos: R$ 1,538.01
Valor médio de operações com Cartão de Crédito de Pessoa Física no País com renda Mais de 5 a 10 salários mínimos: R$ 2,435.37
Valor médio de operações com Cartão de Crédito de Pessoa Física no País com renda Mais de 10 a 20 salários mínimos: R$ 4,123.40
Valor médio de operações com Cartão de Crédito de Pessoa Física no País com renda Acima de 20 salários mínimos: R$ 7,057.66


In [12]:
# Inadimplencia por porte na modalidade cartão de crédito


In [13]:
#df1.loc[:, ['uf','porte','carteira_inadimplida_arrastada','carteira_ativa','modalidade','cliente','numero_de_operacoes']]

df1.head()

,data_base,uf,tcb,sr,cliente,ocupacao,cnae_secao,cnae_subclasse,porte,modalidade,...,a_vencer_ate_90_dias,a_vencer_de_91_ate_360_dias,a_vencer_de_361_ate_1080_dias,a_vencer_de_1081_ate_1800_dias,a_vencer_de_1801_ate_5400_dias,a_vencer_acima_de_5400_dias,vencido_acima_de_15_dias,carteira_ativa,carteira_inadimplida_arrastada,ativo_problematico
0,2022-05-31,AC,Bancário,NaN,PF,Aposentado/pensionista,,,Mais de 1 a 2 salários mínimos,Cartão de crédito,...,3848.70,4691.68,1082.83,0.0,0.0,0.0,10832.38,20455.59,8361.75,8501.76
1,2022-05-31,AC,Bancário,NaN,PF,Aposentado/pensionista,,,Sem rendimento,Outros créditos,...,276.20,171.90,0.00,0.0,0.0,0.0,0.00,448.10,0.00,0.00
2,2022-05-31,AC,Bancário,NaN,PF,Autônomo,,,Mais de 1 a 2 salários mínimos,Cartão de crédito,...,1345.78,660.77,0.00,0.0,0.0,0.0,3698.14,5704.69,3698.14,3698.14
3,2022-05-31,AC,Bancário,NaN,PF,Empregado de empresa privada,,,Mais de 1 a 2 salários mínimos,Cartão de crédito,...,4484.62,7837.42,3971.24,0.0,0.0,0.0,4756.84,21050.12,260.65,1402.21
4,2022-05-31,AC,Bancário,NaN,PF,Empregado de empresa privada,,,Mais de 2 a 3 salários mínimos,Cartão de crédito,...,1973.02,927.40,0.00,0.0,0.0,0.0,0.00,2900.42,0.00,0.00


In [14]:
df1_porte=df1.query('cliente=="PF"')
cols=['porte', 'carteira_ativa', 'carteira_inadimplida_arrastada']
df1_porte=df1_porte[cols].groupby('porte').sum().reset_index()

In [15]:
df1_porte['Inadimplencia'] = (df1_porte['carteira_inadimplida_arrastada']/df1_porte['carteira_ativa'])*100
df1_porte.sort_values('Inadimplencia')

,porte,carteira_ativa,carteira_inadimplida_arrastada,Inadimplencia
0,Acima de 20 salários mínimos,5.705534e+11,3.632652e+09,0.636689
4,Mais de 10 a 20 salários mínimos,3.525617e+11,5.488852e+09,1.556849
7,Mais de 5 a 10 salários mínimos,4.831548e+11,1.250114e+10,2.587398
6,Mais de 3 a 5 salários mínimos,4.163261e+11,1.459234e+10,3.505027
2,Indisponível,1.765288e+11,6.765616e+09,3.832584
5,Mais de 2 a 3 salários mínimos,3.103407e+11,1.390268e+10,4.479814
1,Até 1 salário mínimo,1.604040e+11,9.156820e+09,5.708599
3,Mais de 1 a 2 salários mínimos,4.100096e+11,2.404374e+10,5.864188
8,Sem rendimento,5.912187e+09,1.308781e+09,22.137010


In [16]:
# ---------------------------------------------- Selecionando Operações do Estado de Goiás ------------------------------------------------------------------------#
df_go=df.query('uf=="GO"')

In [17]:
# ---------------------------------------------- Média das operações de PJ  ---------------------------------------------------------------------------------------#

med_pj=(df_go.query('cliente=="PJ"').carteira_ativa.sum())/(df_go.query('cliente=="PJ"').numero_de_operacoes.sum())
print( 'Valor médio de operações de Pessoa Jurídica de Goiás: R$', '{:,.2f}'.format(med_pj))

med_pj=(df.query('cliente=="PJ"').carteira_ativa.sum())/(df.query('cliente=="PJ"').numero_de_operacoes.sum())
print( 'Valor médio de operações de Pessoa Jurídica do País: R$', '{:,.2f}'.format(med_pj))

med_pf=(df_go.query('cliente=="PF"').carteira_ativa.sum())/(df_go.query('cliente=="PF"').numero_de_operacoes.sum())
print( 'Valor médio de operações de Pessoa Física de Goiás: R$', '{:,.2f}'.format(med_pf))

med_pf=(df.query('cliente=="PF"').carteira_ativa.sum())/(df.query('cliente=="PF"').numero_de_operacoes.sum())
print( 'Valor médio de operações de Pessoa Física do País: R$', '{:,.2f}'.format(med_pf))


Valor médio de operações de Pessoa Jurídica de Goiás: R$ 55,075.61
Valor médio de operações de Pessoa Jurídica do País: R$ 65,147.42
Valor médio de operações de Pessoa Física de Goiás: R$ 9,658.64
Valor médio de operações de Pessoa Física do País: R$ 5,658.54


In [18]:
med_pj_giro=(df_go.query('cliente=="PJ"').query('modalidade=="Capital de giro"').carteira_ativa.sum())/(df_go.query('cliente=="PJ"').query('modalidade=="Capital de giro"').numero_de_operacoes.sum())
print( 'Valor médio de operações de Capital de Giro de Pessoa jurídica em Goiás: R$', '{:,.2f}'.format(med_pj_giro))

med_pj_giro=(df.query('cliente=="PJ"').query('modalidade=="Capital de giro"').carteira_ativa.sum())/(df.query('cliente=="PJ"').query('modalidade=="Capital de giro"').numero_de_operacoes.sum())
print( 'Valor médio de operações de Capital de Giro de Pessoa jurídica do País: R$', '{:,.2f}'.format(med_pj_giro))

med_pj_invest=(df_go.query('cliente=="PJ"').query('modalidade=="Investimento"').carteira_ativa.sum())/(df_go.query('cliente=="PJ"').query('modalidade=="Investimento"').numero_de_operacoes.sum())
print( 'Valor médio de operações de Investimento de Pessoa Jurídica em Goiás: R$', '{:,.2f}'.format(med_pj_invest))

med_pj_invest=(df.query('cliente=="PJ"').query('modalidade=="Investimento"').carteira_ativa.sum())/(df.query('cliente=="PJ"').query('modalidade=="Investimento"').numero_de_operacoes.sum())
print( 'Valor médio de operações de Investimento de Pessoa Jurídica em Goiás: R$', '{:,.2f}'.format(med_pj_invest))



Valor médio de operações de Capital de Giro de Pessoa jurídica em Goiás: R$ 101,120.58
Valor médio de operações de Capital de Giro de Pessoa jurídica do País: R$ 130,135.03
Valor médio de operações de Investimento de Pessoa Jurídica em Goiás: R$ 116,273.09
Valor médio de operações de Investimento de Pessoa Jurídica em Goiás: R$ 122,040.89


## Calculando a Inadimplência de Pessoa Jurídica e Física


In [19]:
# --------------------------------------------- Calculando inadimplência do Estado para Operações de clientes PJ -------------------------------------#
# Somatória da coluna Carteira_Inadimplida_arrastada dividida pela Somatória da coluna carteira_ativa
inadimpj = df_go.query('cliente=="PJ"').carteira_inadimplida_arrastada.sum()/df_go.query('cliente=="PJ"').carteira_ativa.sum()
print( 'Inadimplência em operações de Pessoa jurídica em Goiás: ', '{:,.2f} %'.format(inadimpj*100))

inadimpj = df.query('cliente=="PJ"').carteira_inadimplida_arrastada.sum()/df.query('cliente=="PJ"').carteira_ativa.sum()
print( 'Inadimplência em operações de Pessoa jurídica no País: ', '{:,.2f} %'.format(inadimpj*100))


inadimpf = df_go.query('cliente=="PF"').carteira_inadimplida_arrastada.sum()/df_go.query('cliente=="PF"').carteira_ativa.sum()
print( 'Inadimplência em operações de Pessoa física em Goiás: ', '{:,.2f} %'.format(inadimpf*100))


inadimpf = df.query('cliente=="PF"').carteira_inadimplida_arrastada.sum()/df.query('cliente=="PF"').carteira_ativa.sum()
print( 'Inadimplência em operações de Pessoa física no País: ', '{:,.2f} %'.format(inadimpf*100))


Inadimplência em operações de Pessoa jurídica em Goiás:  1.35 %
Inadimplência em operações de Pessoa jurídica no País:  1.16 %
Inadimplência em operações de Pessoa física em Goiás:  2.53 %
Inadimplência em operações de Pessoa física no País:  3.17 %


### Explorando alguns visuais

In [20]:
cols=['porte', 'carteira_ativa', 'carteira_inadimplida_arrastada']
df1 = df_go.query('cliente=="PJ"').loc[:, cols].groupby('porte').sum().reset_index()
df1['inadimplencia %']= round(df1['carteira_inadimplida_arrastada']*100/df1['carteira_ativa'],2)
fig = px.bar(df1, x='porte', y='inadimplencia %', title=" Goiás - Inadimplencia por Porte - PJ")
fig

In [21]:
cols=['porte', 'carteira_ativa', 'carteira_inadimplida_arrastada']
df1 = df_go.query('cliente=="PF"').loc[:, cols].groupby('porte').sum().reset_index()
df1['inadimplencia %']= round(df1['carteira_inadimplida_arrastada']*100/df1['carteira_ativa'],2)
fig = px.bar(df1, x='porte', y='inadimplencia %',title=" Goiás - Inadimplencia por Porte - PF")
fig

In [22]:
cols=['modalidade', 'carteira_ativa', 'carteira_inadimplida_arrastada']
df1 = df_go.query('cliente=="PF"').loc[:, cols].groupby('modalidade').sum().reset_index()
df1['inadimplencia %']= round(df1['carteira_inadimplida_arrastada']*100/df1['carteira_ativa'],2)
fig = px.bar(df1, x='modalidade', y='inadimplencia %',title="Goiás - Inadimplencia por Modalidade - PF")
fig

In [23]:
cols=['modalidade', 'carteira_ativa', 'carteira_inadimplida_arrastada']
df1 = df_go.query('cliente=="PJ"').loc[:, cols].groupby('modalidade').sum().reset_index()
df1['inadimplencia %']= round(df1['carteira_inadimplida_arrastada']*100/df1['carteira_ativa'],2)
fig = px.bar(df1, x='modalidade', y='inadimplencia %',title="Goiás - Inadimplencia por Modalidade - PJ")
fig